In [ ]:
from gatsbi.task_utils.benchmarks import make_generator, make_discriminator
from gatsbi.optimize import Base as Opt
import sbibm

In [ ]:
task_name = "two_moons"
task = sbibm.get_task(task_name)
prior = task.get_prior()
simulator = task.get_simulator()
